In this example, we relax the phase retrieval problem similar to the classical [MaxCut](http://www-math.mit.edu/~goemans/PAPERS/maxcut-jacm.pdf) semidefinite program and recover the phase of the signal given the magnitude of the linear measurements.

Phase recovery has wide applications such as  in X-ray and crystallography imaging, diffraction imaging or microscopy and audio signal processing. In all these applications, the detectors cannot measure the phase of the incoming wave and only record its amplitude i.e complex measurements of a signal $x \in \mathbb{C}^p$ are obtained from a linear injective operator A, **but we can only measure the magnitude vector |Ax| not the phase fo Ax**.

Recovering the phase of Ax from |Ax| is a **nonconvex optimization problem**. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

The original reprsentation of the problem is as follows:

>>>> find x

>>>> such that |Ax| = b

>>>> where $x \in \mathbb{C}^p$, $A \in \mathbb{C}^{n×p}$ and $b \in \mathbb{R}^n$.



In this example, **the problem is to find the phase of Ax given the value |Ax|**. Given a linear operator $A$ and a vector $b= |Ax|$ of measured amplitudes, in the noiseless case, we can write Ax = diag(b)u where $u \in \mathbb{C}^n$ is a phase vector, satisfying |$\mathbb{u}_i$| = 1 for i = 1,. . . , n. 

We relax this problem as Complex Semidefinite Programming.

### Relaxed Problem similar to [MaxCut](http://www-math.mit.edu/~goemans/PAPERS/maxcut-jacm.pdf)

Define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.

In [94]:
using Convex

n = 10
p = 2
A = rand(n,p) + im*randn(n,p)
x = rand(p) + im*randn(p)
b = abs(A*x) + rand(n)

M = diagm(b)*(eye(n)-A*ctranspose(A))*diagm(b)
U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((20,20),(20,20))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 791
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 201, constraints m = 421
Cones:	primal zero / dual free vars: 211
	sd vars: 210, sd blks: 1
Setup time: 6.96e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  6.46e-03 
   100| 3.46e-03  8.72e-02  1.06e-06 -1.02e+02 -1.02e+02  1.16e-15  1.23e-01 
   200| 2.66e-04  1.89e-02  8.20e-06 -1.02e+02 -1.02e+02  6.43e-16  1.56e-01 


10×10 Array{Complex{Float64},2}:
 1.0+3.7352e-17im             0.691233+0.722632im  …      0.687651-0.726041im
    0.691233-0.722632im   1.0-2.50157e-16im             -0.0493335-0.998782im
    0.996785+0.080119im       0.631115+0.77569im           0.74361-0.668614im
    0.278011+0.960578im      -0.501974+0.864883im         0.888594+0.458695im
    0.979366-0.202097im       0.823012+0.568025im         0.526731-0.850032im
     0.99584+0.0911115im      0.622518+0.782605im  …      0.750941-0.660368im
    0.840265-0.542176im       0.972613+0.232433im         0.184167-0.982895im
   -0.242079+0.970256im      -0.868472+0.495739im          0.53798+0.842958im
    0.999503+0.0315263im      0.668108+0.744065im         0.710199-0.704001im
    0.687651+0.726041im     -0.0493335+0.998782im     1.0-3.33738e-16im      

In [95]:
# Verify if the rank of U is 1
B, C = eig(U.value);
print(length([e for e in B if(abs(real(e))>1e-4)]))
#Decompose U = uu*
# u is the phase of Ax
u = C[:,1]

1

10-element Array{Complex{Float64},1}:
   0.315211-0.0253359im
   0.199576-0.245295im 
   0.316228+0.0im      
   0.111969+0.295741im 
   0.303587-0.0885164im
   0.316208+0.0034889im
   0.251125-0.192189im 
 -0.0517238+0.311969im 
   0.315853-0.0153858im
    0.23515+0.211434im 

In [96]:
pi = π

π = 3.1415926535897...

In [97]:
phase_u = atan(sum(imag(u)/sum(real(u))))

0.11018387297527316

In [98]:
phase_Ax = atan(sum(imag(A*x)/sum(real(A*x))))

0.19297018950357864

In [99]:
# 
norm(abs(phase_u-phase_Ax),Inf)

0.08278631652830548